# Imports

In [1]:
#export
import os
import sys
sys.path.append(os.path.join(os.getcwd(),'exp'))

import yaml
import json
import numpy as np
from pprint import pprint
from skopt import Optimizer
from skopt.space import Real, Integer
from collections import Mapping, defaultdict, OrderedDict

from config import cfg

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

# Code

In [3]:
#export
def dict_merge(dct, merge_dct):
    for k, v in merge_dct.items():
        if (k in dct and isinstance(dct[k], dict)
                and isinstance(merge_dct[k], Mapping)):
            dict_merge(dct[k], merge_dct[k])
        else:
            dct[k] = merge_dct[k]

def create_nested(k, v, sep='|'):
    d = defaultdict(dict)
    nested = d
    for i in k.split(sep)[:-1]:
        nested.setdefault(i, {})
        nested = nested[i]
    nested[k.split('|')[-1]] = v
    return d

In [4]:
#export
class BaseConfigBo:
    def __init__(self, n):
        self.n_points = n
        
    def read_params(self, params):
        bounds = {}
        for name, settings in params.items():
            if settings['default'] is not None: continue
            space_args = {}
            space_type = Real if settings['type'] is float else Integer
            space_args['low'],space_args['high'] = settings['bounds'][0], settings['bounds'][1]
            space_args['prior'] = settings['prior']
            bounds[name] = space_type(**space_args)
        
        bounds = [bounds[k] for k in sorted(bounds)]
        return bounds
    
    def init_opt(self, bounds):
        return Optimizer(
                    dimensions=bounds,
                    random_state=1,
                    base_estimator='gp',
                    n_initial_points=2*self.n_points)
        
    def register(self, o, hp_points, boparams):
        NEGATIVE = -1
        for hp_point in hp_points:
            points = hp_point['points']
            x = []
            for k1, k2 in zip(sorted(points), boparams):
                if k1 == k2: x.append(points[k1])
                else:
                    print('keys mismatch: ', k1, k2)
                    raise ValueError
            y = NEGATIVE * hp_point['target']
            try:
                o.tell(x, y)
                print(f'Registrating: {x}, {y}')
            except ValueError as v:
                print(f'\n\n\tWARNING, point is out of bounds: {v}')
        
    def get_values(self, o, names):
        o.update_next()
        set_points = o.ask(n_points=self.n_points)
        all_points = []
        for points in set_points:
            ps = {}
            for n,p  in zip(names, points):
                t = self.params[n]['type']
                ps[n] = t(p)
            all_points.append(ps)   
        #points = [{n:p for n,p in zip(names, points)} for points in set_points]
        return all_points
    
    def init_warmups(self,  warm_list, boparams):
        warmups = []
        if warm_list:
            for w in warm_list:
                warmups.extend(self.read_warmup(w, boparams))
        return warmups
    
    def read_warmup(self, warmup, boparams):
        if not warmup:
            return []
        with open(warmup, 'r') as f:
            data = json.load(f)
        cleared_data = []
        for hppoint in data:
            points = hppoint['points']
            hpkeys = set(points.keys())
            if hpkeys.intersection(boparams) != set(boparams):
                return cleared_data
            
            cleared_point = {}
            for p in boparams:
                cleared_point[p] = points[p]
            cleared_data.append({'points':cleared_point, 'target':hppoint['target']})
        return cleared_data
    
    def create_state(self, points, idx):
        points = points.copy()
        o = self.init_opt(self.bounds)
        points.extend(self.warmup)
        if points: self.register(o, points, self.boparams)
        
        list_of_new_params = self.get_values(o, self.boparams)
        new_params = list_of_new_params[idx]
        pprint(new_params, indent=4)
        
        cfg = {}
        for name, settings in self.params.items():
            new_value = new_params[name] if not settings['default'] else settings['default']
            sub_cfg = create_nested(name, settings['type'](new_value))
            dict_merge(cfg, sub_cfg)
        
        return new_params, cfg
    
class Bo(BaseConfigBo):
    def __init__(self, params, warm_list, *args, **kwargs):
        super(Bo, self).__init__(*args, **kwargs)
        self.params =  OrderedDict(sorted(params.items(), key=lambda x:x[0], reverse=False))
        self.boparams = [k for k in self.params if self.params[k]['default'] is None]
        self.bounds = self.read_params(self.params)
        self.warmup = self.init_warmups(warm_list, self.boparams)    

# Tests

In [35]:
# points = [{'params':{'e':1,  'a':2},'target':0.1},
#           {'params':{'e':1,  'a':2},'target':0.2},
#           {'params':{'e':1,'a':2},'target':0.5},
#           {'params':{'e':1.2,'d':2},'target':0.0},
#           {'params':{'e':330,'k':2},'target':0.2}
#          ]

points = [
    {'params': {'cr': 0.9885257859302491}, 'target': 0.577763631939888},
    {'params': {'cr': 0.9885660456872435}, 'target': 0.5584937930107117},
    {'params': {'cr': 0.010119437482166772}, 'target': 0.5518008545041084},
    {'params': {'cr': 0.0101559066291012}, 'target': 0.5431795716285706},
    {'params': {'cr': 0.9899404424902588}, 'target': 0.5732786804437637},
    {'params': {'cr': 0.9893949599413431}, 'target': 0.5737223923206329},
    {'params': {'cr': 0.9898670833454175}, 'target': 0.5675030797719955},
    {'params': {'cr': 0.3092491872609939}, 'target': 0.5605851411819458},
    {'params': {'cr': 0.6836879323470609}, 'target': 0.567135363817215},
    {'params': {'cr': 0.5175112773164423}, 'target': 0.5583183616399765},
    {'params': {'cr': 0.010109605687386128}, 'target': 0.5333066955208778}
]

In [36]:
bounds = [Real(10,15, transform='normalize')]

In [26]:
o = Optimizer(   dimensions=bounds,
            random_state=1,
            base_estimator='gp',
            n_initial_points=3)

In [22]:
o.ask()

[14.231554583430086]

In [18]:
o.tell([11.982903636480131], 1)

          fun: 1
    func_vals: array([1])
       models: []
 random_state: RandomState(MT19937) at 0x7F9C8F01F048
        space: Space([Real(low=10, high=15, prior='uniform', transform='normalize')])
        specs: None
            x: [11.982903636480131]
      x_iters: [[11.982903636480131]]

In [65]:
from random import shuffle

In [66]:
p1 = {'name':'cr', 'bounds':(0.01,.99)}
shuffle(points)

# More Tests

In [7]:
#export
n_parallel_processes = len(cfg.GPUS.IDS)
warm_list = ['/home/sokolov/work/cycler/dHPO/2020_May_21_18_34_23_hp.json']

params_static = {
    'generations':{'default':200, 'type':int},
}
params_genom = {
            'genom|mutate_chance':{'bounds':(0,.05), 'type':float, 'prior':'uniform', 'default':None},
            'genom|crossover_chance':{'bounds':(0,1), 'type':float, 'prior':'uniform', 'default':None},
            'genom|combine_chance':{'bounds':(0,1), 'type':float, 'prior':'uniform', 'default':None},
            }
params_post = {
    'post|exp_power':{'bounds':(1,15), 'type':int, 'prior':'uniform', 'default':None},
}

p_all = {}
[dict_merge(p_all, d) for d in [params_static, params_genom, params_post]]
b1 = Bo(n=n_parallel_processes, params=p_all, warm_list=warm_list)

b2 = Bo(n=n_parallel_processes, params=params_post, warm_list=warm_list)


def bo_all(**kwargs):
    inner_state, new_state=bo_all.create_state(points=kwargs['hp_points'], idx=kwargs['idx'])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)

def bo_exp(**kwargs):
    inner_state, new_state=bo_exp.create_state(points=kwargs['hp_points'], idx=kwargs['idx'])
    return run(new_state=new_state, inner_state=inner_state, **kwargs)

In [9]:
b1

In [11]:
points = [{'points':{'genom|crossover_chance': 0.493, 'post|exp_power': 1.14, 'genom|combine_chance':.2}, 'target':0.3},
         { 'points':{'genom|crossover_chance': 0.6985, 'post|exp_power': 1.1, 'genom|combine_chance':.9}, 'target':.4},
           { 'points':{'genom|crossover_chance': 0.67, 'post|exp_power': 1.5, 'genom|combine_chance':.5}, 'target':.4},
         ]

# points = [{'points':{'cr': 0.493, 'mc': .014, 'co':.2}, 'target':0.3},
#          { 'points':{'cr': 0.6985, 'mc': .01, 'co':.9}, 'target':.4},
#            { 'points':{'cr': 0.67, 'mc': .5, 'co':.5}, 'target':.4},
#          ]
points = [{"points":{   'genom|combine_chance': 0.702644992444359,
    'genom|crossover_chance': 0.061651731814223246,
    'genom|mutate_chance': 0.01884459380869284,
    'post|exp_power': 3}, 'target':.1}]
af = {
    'hp_points':points,
    'idx':0
}

In [12]:
n_parallel_processes = len(cfg.GPUS.IDS)
warm_list = None

b2 = Bo(n=n_parallel_processes, params=params_post, warm_list=warm_list)

In [13]:
ns, ccfg = b1.create_state(points=af['hp_points'], idx=af['idx'])

Registrating: [0.702644992444359, 0.061651731814223246, 0.01884459380869284, 3], -0.1
Registrating: [0.702644992444359, 0.061651731814223246, 0.01884459380869284, 3], -0.47418662160634995
Registrating: [0.49317547373314585, 0.8889254049717007, 0.0029243592954225524, 5], -0.5729198455810547
Registrating: [0.47895657759933064, 0.6760874977543353, 0.015782121348437275, 15], -0.542678639292717
Registrating: [0.702644992444359, 0.061651731814223246, 0.01884459380869284, 3], -0.4781750291585922
Registrating: [0.49317547373314585, 0.8889254049717007, 0.0029243592954225524, 5], -0.574361115694046
Registrating: [0.47895657759933064, 0.6760874977543353, 0.015782121348437275, 15], -0.5501270592212677
{   'genom|combine_chance': 0.5173994830665458,
    'genom|crossover_chance': 0.5100114319247788,
    'genom|mutate_chance': 0.047197218994194205,
    'post|exp_power': 2}


In [52]:
with open('t.yaml', 'w') as f:
    json.dump(ns, f)

In [11]:
with open('t.yaml', 'w') as f:
    yaml.safe_dump(ccfg, f)

In [ ]:
for i in range(10):
    x = optimizer.ask(n_points=4)  # x is a list of n_points points
    y = Parallel(n_jobs=4)(delayed(branin)(v) for v in x)  # evaluate points in parallel
    optimizer.tell(x, y)

# takes ~ 20 sec to get here
print(min(optimizer.yi))  # print the best objective found

# Exports

In [14]:
!python3 extra/n2s.py bo.ipynb

Converted bo.ipynb to exp/nb_bo.py
